In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": [],
      "gpuType": "T4",
      "include_colab_link": true
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    },
    "accelerator": "GPU"
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "view-in-github",
        "colab_type": "text"
      },
      "source": [
        "<a href=\"https://colab.research.google.com/github/chironhooves/multilingual_tts_system/blob/main/Multilingual_TTS_Colab_Setup.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "header"
      },
      "source": [
        "# 🎤 Multilingual TTS System - Google Colab Setup\n",
        "\n",
        "## Features:\n",
        "- ✅ **10 Indian Languages** (Hindi, Tamil, Telugu, Bengali, Marathi, Gujarati, Kannada, Malayalam, Punjabi, Odia)\n",
        "- ✅ **Legal Open Datasets** (Mozilla Common Voice, Google FLEURS, OpenSLR)\n",
        "- ✅ **Advanced Speaker Diarization** (Custom implementation)\n",
        "- ✅ **TPU/GPU Acceleration** \n",
        "- ✅ **Real-time Visualization** \n",
        "- ✅ **Professional Quality Output**\n",
        "\n",
        "---\n",
        "\n",
        "## 🚀 Quick Start Guide:\n",
        "1. **Run all cells** in order (Runtime → Run all)\n",
        "2. **Select your languages** when prompted\n",
        "3. **Start training** and monitor progress\n",
        "4. **Download your models** when complete\n",
        "\n",
        "---"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "setup"
      },
      "source": [
        "## 🔧 Step 1: Environment Setup & Hardware Check"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "hardware_check"
      },
      "outputs": [],
      "source": [
        "# Check hardware and setup environment\n",
        "import os\n",
        "import sys\n",
        "import torch\n",
        "import time\n",
        "import numpy as np\n",
        "\n",
        "# Record start time\n",
        "training_start_time = time.time()\n",
        "\n",
        "print(\"🔍 Hardware & Environment Check\")\n",
        "print(\"=\" * 40)\n",
        "\n",
        "# Check GPU/TPU availability\n",
        "if torch.cuda.is_available():\n",
        "    print(f\"✅ GPU Available: {torch.cuda.get_device_name(0)}\")\n",
        "    print(f\"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB\")\n",
        "    device_type = \"GPU\"\n",
        "else:\n",
        "    print(\"⚠️  No GPU detected\")\n",
        "    device_type = \"CPU\"\n",
        "\n",
        "# Check for TPU\n",
        "try:\n",
        "    import torch_xla\n",
        "    import torch_xla.core.xla_model as xm\n",
        "    if xm.xla_device_hw(xm.xla_device()) == 'TPU':\n",
        "        print(\"✅ TPU Available!\")\n",
        "        device_type = \"TPU\"\n",
        "        os.environ['XLA_USE_BF16'] = '1'\n",
        "        os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'\n",
        "except ImportError:\n",
        "    print(\"ℹ️  TPU libraries not available\")\n",
        "\n",
        "print(f\"\\n🎯 Selected Device: {device_type}\")\n",
        "print(f\"📍 Python Version: {sys.version}\")\n",
        "print(f\"📁 Working Directory: {os.getcwd()}\")\n",
        "\n",
        "# Set environment variables\n",
        "os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'\n",
        "os.environ['TOKENIZERS_PARALLELISM'] = 'false'\n",
        "\n",
        "print(\"\\n✅ Environment configured for optimal performance!\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "clone_repo"
      },
      "source": [
        "## 📦 Step 2: Clone Repository & Install Dependencies"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "clone_install"
      },
      "outputs": [],
      "source": [
        "# Clone repository and install dependencies\n",
        "!echo \"📥 Cloning Multilingual TTS Repository...\"\n",
        "!git clone https://github.com/chironhooves/multilingual_tts_system.git\n",
        "%cd multilingual_tts_system\n",
        "\n",
        "# Install system dependencies\n",
        "!echo \"\\n🔧 Installing system dependencies...\"\n",
        "!apt-get update -qq\n",
        "!apt-get install -y -qq ffmpeg sox libsox-fmt-all portaudio19-dev python3-pyaudio\n",
        "\n",
        "# Install Python dependencies\n",
        "!echo \"\\n📚 Installing Python packages...\"\n",
        "!pip install -q torch torchaudio numpy scipy librosa soundfile matplotlib plotly ipywidgets\n",
        "\n",
        "# Install TPU packages if available\n",
        "if device_type == \"TPU\":\n",
        "    !pip install -q torch-xla[tpu]\n",
        "    print(\"✅ TPU packages installed\")\n",
        "\n",
        "# Verify installation\n",
        "print(\"\\n🧪 Verifying installation...\")\n",
        "try:\n",
        "    import torch, librosa, soundfile, matplotlib, plotly\n",
        "    print('✅ All packages working correctly')\n",
        "except ImportError as e:\n",
        "    print(f'⚠️  Some packages missing: {e}')\n",
        "\n",
        "print(\"\\n🎉 Installation completed successfully!\")"
      ]
    }
  ]
}
